# ByteNite Guide: Storj integration
Storj I/O - storage configuration

---
In this notebook, you can set up Storj's storage feature for encoding jobs using the ByteNite Encoding API. You have the option to import your videos and store the output in a pre-existing Storj bucket.

_Note: Have you signed up on Storj yet? If not, visit [storj.io/signup](storj.io/signup) to create an account and obtain your S3 Credentials, which include your access and secret key. These credentials will allow you to access the necessary buckets._

_Note: This notebook shows how to acquire and store transcoded videos on your Storj DCS. The notebook can be customized to utilize either of the two options (e.g., using Storj for the output, while getting the input from GCS)._


<h1 style="color:blue">Setup (edit with your info)</h1>

## Your ByteNite API Key ↓
Get it at https://app.bytenite.com/profile

In [1]:
# >> Paste here your API key (Go to app.bytenite.com/profile)
access_token = 'BYTENITE_ACCESS_TOKEN'

## Your Storj's S3 Credentials ↓
Get them at https://us1.storj.io/access-grants/create-access-modal

In [2]:
# >> Paste here your access key
storj_S3_access = 'ACCESS_KEY'

# >> Paste here you secret key
storj_S3_secret = 'SECRET_KEY'

## Your Storj input ↓


In [3]:
# >> Please enter the name of the existing Storj bucket where your source video is stored
storj_source_bucket = "input-bytenite"

# >> Please enter the file path for your source video
storj_source_path = "/bbb.mp4"

# >> Please specify your source cloud region
storj_source_region = "us1"

## Your Storj output ↓


In [1]:
# >> Please enter the name of the existing Storj bucket where you would like your output video to be saved
storj_dest_bucket = "output-bytenite"

# >> Please provide the file path for the FOLDER where you would like to save your video.
# In case the folder does not exist, it will be generated automatically.
storj_dest_path = "/"

# >> Please specify your destination cloud region
storj_dest_region = "us1"

<h1 style="color:orange">Additional setup (edit if you need different encoding options)</h1>
The default encoding parameters for this notebook are:

- HLS H.265 360p @365k
- HLS H.265 720p @4.5M

You can configure your parameters from the "New Job" wizard on app.bytenite.com, and then copy-paste the document by clicking on "Switch to JSON" on the top right.

_Note: "true" / "false" values must be changed to Python's "True" / "False" (capital)_

In [14]:
## Job parameters dictionary (JSON)
job_params_body={
    "data":{"output_code_template":"","output_template":"{{if .name}}{{.name}}{{else}}{{.job_name}}_{{.output_params.Aspect.Resolution.CustomHeight}}p@{{.output_params.Video.Bitrate.AvgBitrate}}{{end}}",
            "outputs":[
                {"name":"360p @365kbps","output_params":{"aspect":{"cropping":{"bottom":0,"left":0,"right":0,"top":0},"orientation":{"flip":"","rotation":""},"padding":{"fill_color":"#000000","final_aspect_ratio":""},"resolution":{"aspect_ratio":"","avoid_upscaling":True,"custom_height":360,"height":"custom"}},"audio":{"audio_bitrate":"160k","audio_channel":"2.0","audio_codec":"AAC","audio_sample_rate":"48khz"},"video":{"bitrate":{"avg_bitrate":"365k","buffer_size":"548k","max_bitrate":"548k","min_bitrate":"548k","rate_control_mode":"variable_bitrate"},"codec_params":{"level":"","preset":"medium","profile":"high","tune":""},"frame_rate":{"fps":"30","up_mode":"avoid"},"codec":"libx264"}},"output_type":"hls"},
                {"name":"720p @4.5Mbps","output_params":{"aspect":{"cropping":{"bottom":0,"left":0,"right":0,"top":0},"orientation":{"flip":"","rotation":""},"padding":{"fill_color":"#000000","final_aspect_ratio":""},"resolution":{"aspect_ratio":"","avoid_upscaling":True,"custom_height":720,"height":"custom"}},"audio":{"audio_bitrate":"160k","audio_channel":"2.0","audio_codec":"AAC","audio_sample_rate":"48khz"},"video":{"bitrate":{"avg_bitrate":"4500k","buffer_size":"6750k","max_bitrate":"6750k","min_bitrate":"6750k","rate_control_mode":"variable_bitrate"},"codec_params":{"level":"","preset":"medium","profile":"high","tune":""},"frame_rate":{"fps":"60","up_mode":"avoid"},"codec":"libx264"}},"output_type":"hls"}
            ]}}



-------



<h1 style="color:red">Execution (do not alter)</h1>

In [15]:
# Imports
import string
import requests, os, time

<h2 style="color:red">Job creation and submission via ByteNite API</h2>

In [16]:
# JOBS CREATION

## Data source and data destination definition
data_source_body={
    "dataSource": {
        "dataSourceDescriptor":"storj",
        "params":{
          "@type": "type.googleapis.com/bytenite.data_source.StorjDataSource",
          "accessKey": storj_S3_access,
          "secretKey": storj_S3_secret,
          "cloudRegion": storj_source_region,
          "bucketName": storj_source_bucket,
          "name": storj_source_path
        }
    },
    "dataDestination": {
        "dataSourceDescriptor":"storj",
        "params":{
          "@type": "type.googleapis.com/bytenite.data_source.StorjDataSource",
          "accessKey": storj_S3_access,
          "secretKey": storj_S3_secret,
          "cloudRegion": storj_dest_region,
          "bucketName": storj_dest_bucket,
          "name": storj_dest_path
        }
    }
}

# Custom name
job_name = storj_source_path.split('/')[-1]
job_name = "storj-" + job_name[:job_name.find(".")]

# CREATE JOB
create_job_body = {"name": job_name, "templateId": "video-transcoding@v0.2"}
create_resp = requests.post('http://api.bytenite.com/v1/customer/jobs', json=create_job_body, headers={'Authorization': access_token}).json()
try:
    job_data = create_resp['job']
    job_id = job_data['id']
    print(f"Job \"{job_name}\" created - ID: {job_id}")
except:
    print(f"CREATE ERROR")
    print(create_resp)
               
# SET SOURCE AND DESTINATION
datasource_resp = requests.post(f'https://api.bytenite.com/v1/customer/jobs/datasource/{job_id}', json=data_source_body, headers={'Authorization': access_token}).json()
try:
    job_data = datasource_resp['job']
    temp_url = job_data['dataSource']['params']['tempUrl']
except:
    print(f"DATASOURCE ERROR")
    print(datasource_resp)
    
# SET PARAMETERS
params_resp = requests.post(f'https://api.bytenite.com/v1/customer/jobs/params/{job_id}', json=job_params_body, headers={'Authorization': access_token}).json()
try:
    job_data = params_resp['job']
except:
    print(f"PARAMETERS SETTING ERROR")
    print(params_resp)
        
time.sleep(0.5)
    
# RUN JOB
run_resp = requests.post(f'https://api.bytenite.com/v1/customer/jobs/run/{job_id}', json={}, headers={'Authorization': access_token}).json()
try:
    job_started = run_resp['ok']
    print("Job started!")
except:
    print(f"RUN JOB ERROR")
    print(run_resp)


Job "storj-bbb" created - ID: 21a56ebf-0eff-44f7-944d-677b3e80a47b
Job started!


-------


<h1 style="color:green">Get the results (run after launching the jobs)</h1>

In [19]:
import dateutil.parser
from IPython.display import display, HTML

results_resp = requests.get(f'https://api.bytenite.com/v1/customer/jobs/{job_id}/results', headers={'Authorization': access_token}).json()
print(f"{job_name} - Outputs")
try:
    results_data = results_resp['results']
    job_data = requests.get(f'https://api.bytenite.com/v1/customer/jobs/{job_id}', json=job_params_body, headers={'Authorization': access_token}).json()['job']
    job_time = (dateutil.parser.isoparse(job_data['completedAt']) - dateutil.parser.isoparse(job_data['startedAt'])).total_seconds()
    print(f"Running time: {int(job_time)} seconds")
    for j in range(max(len(results_data),1)):
        display(HTML(f"<a href='{results_data[j]['link']}' target='_blank'>Output {j+1}: Storj authenticated link</a>"))
except KeyError:
    print(f"ERROR: {results_resp}")
except (IndexError, TypeError):
    display(HTML(f"<b>Not ready</b> (wait and rerun cell)..."))

storj-bbb - Outputs


## Troubleshooting

For more info, visit https://github.com/ByteNite2/bytenite-api-docs or contact us at support@bytenite.com